In [2]:

import os
import numpy as np
import pandas as pd
import scipy
from itertools import combinations
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from importlib import reload
import networkx as nx
import Levenshtein 

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm

# local imports
import file_loader_funcs as _load
import distance_funcs as _dist
import plotting_funcs as _plot

%matplotlib inline

In [3]:
""" Define a sample """

reload(_load)

SAMPLE_SIZE = 2000
ROOT_DIR = "/Volumes/Cooper_TB_Drive/research/rajapakse/b_cell_1/public-bcell-dataset/"
# DATASETS = [
#     'D1-M',
#     'D1-Na',
#     'D1-Nb',
#     'D2-N',
#     'D2-M',
#     'D3-M',
#     'D3-N',
# ]


DATASETS = [
    'D3-M',
    'D3-N',
]


data = {}

for dataset in DATASETS:
    DIRPATH = f"{ROOT_DIR}{dataset}"
    df = _load.get_samples(DIRPATH, n_sequences=SAMPLE_SIZE)
    data[dataset] = {'sample_df' : df}
    print(dataset, df.shape)

D3-M (2000, 75)
D3-N (2000, 75)


In [4]:
""" filter cdr3 region. NOTE: this is done by slicing the sequence
from the start of the v region `n` nucleotides based on the reported cdr3
length column """

def _apply_crd3_slice(row):
    """An apply function to extrasct the cdr3 region from the 
    sequence""" 
    seq = row['nucleotide']
    start = row['vIndex']
    end = start + row['cdr3Length']
    cdr3 = seq[start:end]
    return cdr3


# define the column in the sampled dataframe 
# for each sample
for k, v in data.items():
    v['sample_df']['cdr3_sequence'] = v['sample_df'].apply(lambda row: _apply_crd3_slice(row), axis=1)

In [5]:
"""Compute distance matrix using real minimal edit distance
from the python package leveshtien. Build a graph object """

for k, v in data.items():
    A = _dist.matrix_levenshtien(v['sample_df']['cdr3_sequence'].tolist())
    data[k]['distance_matrix'] = A
    print(k, A.shape)

D3-M (2000, 2000)
D3-N (2000, 2000)


In [6]:
"""Create network objects from these distance matrices"""

for k, v in data.items():
    g = nx.from_numpy_matrix(data[k]['distance_matrix'])
    g.pos = nx.spring_layout(g, weight='weight')
    data[k]['graph'] = g

    print(dir(g))



['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adj', '_node', 'add_edge', 'add_edges_from', 'add_node', 'add_nodes_from', 'add_weighted_edges_from', 'adj', 'adjacency', 'adjlist_inner_dict_factory', 'adjlist_outer_dict_factory', 'clear', 'copy', 'degree', 'edge_attr_dict_factory', 'edge_subgraph', 'edges', 'get_edge_data', 'graph', 'graph_attr_dict_factory', 'has_edge', 'has_node', 'is_directed', 'is_multigraph', 'name', 'nbunch_iter', 'neighbors', 'node_attr_dict_factory', 'node_dict_factory', 'nodes', 'number_of_edges', 'number_of_nodes', 'order', 'pos', 'remove_edge', 'remove_edges_from', 'remove_node', 'remove_nodes_from', 'size', 'su

In [6]:
# """ Try graph viz: WARNING: slow execution time """

# matplotlib.rcParams['figure.dpi'] = 300
# matplotlib.rcParams['figure.figsize'] = 18, 18
# plt.style.use('seaborn-deep')
# _plot.plot_network_centrality(g)

# plt.suptitle(f"{DATA_NAME} Sample Network", fontsize=30)
# outpath = f"figures/{DATA_NAME}_by_information_centrality_sample.png"
# plt.savefig(outpath, bbox_inches='tight')


In [7]:
"""
Repreated samples with replacement to see the distributions of node centrality
"""

# N_SAMPLES = 30
# SAMPLE_SIZE = 100

# matplotlib.rcParams['figure.dpi'] = 300
# matplotlib.rcParams['figure.figsize'] = 7, 7
# plt.style.use('seaborn-deep')

# for i in range(N_SAMPLES):
#     sample_inds = np.random.choice(list(range(A.shape[0])), SAMPLE_SIZE, replace=False)
#     sample_A = A[sample_inds][:,sample_inds]
#     sample_g = nx.from_numpy_matrix(sample_A)
#     centralities = nx.eigenvector_centrality(sample_g, weight='weight')
#     centralities = list(centralities.values())
#     sns.kdeplot(centralities, color=(0.1, 0.3, 0.5, 0.2))

# plt.suptitle(f"{DATA_NAME} Eigenvector Centrality Distributions")
# plt.xlabel("Eigenvector centrality")
# outpath = f"figures/{DATA_NAME}_centrality_distributions.png"
# plt.savefig(outpath, bbox_inches='tight')

'\nRepreated samples with replacement to see the distributions of node centrality\n'